# Base Model
* 피처엔지니어링 없이 기본적인 스코어 확인

## 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
d = pd.read_csv('./data/sub.csv')

In [5]:
d.target = -8

In [7]:
d.to_csv('./data/sub.csv')

## 데이터 로드

In [3]:
path = './data/'

In [24]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
# history = pd.read_csv(path + 'historical_transactions.csv')
# new_history = pd.read_csv(path + 'new_merchant_transactions.csv')
# merchant = pd.read_csv(path + 'merchants.csv')

## 데이터 선처리

In [8]:
agg_history = history.groupby('card_id').mean()

In [25]:
train = pd.merge(train, agg_history, on='card_id' ,how='left')
test = pd.merge(test, agg_history, on='card_id' ,how='left')

In [26]:
train.set_index('card_id', inplace=True)
test.set_index('card_id', inplace=True)

In [27]:
train = train.drop(train.dtypes[train.dtypes == object].index, axis=1)
test = test.drop(test.dtypes[test.dtypes == object].index, axis=1)

In [28]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

## 모델 모듈 임포트

In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [46]:
lgb = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                    importance_type='split', learning_rate=0.1, max_depth=8,
                    min_child_samples=20, min_child_weight=0.001,
                    min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
                    random_state=0)

In [47]:
lr = LinearRegression()

In [48]:
cv = KFold(5, shuffle=True, random_state=0)

In [49]:
for i, (idx_train, idx_cv) in enumerate(cv.split(train)):
    df_train = train.iloc[idx_train]
    df_cv = train.iloc[idx_cv]
    
    x_train = df_train.drop(['target'], axis=1)
    y_train = df_train['target']
    
    x_cv = df_cv.drop(['target'], axis=1)
    y_cv = df_cv['target']
    
    model = lgb
    model.fit(x_train, y_train)
    print("[K = {}] train rmse = {}, cv_rmse = {}".format(i, np.sqrt(np.mean((model.predict(x_train) - y_train) ** 2)), np.sqrt(np.mean((model.predict(x_cv) - y_cv) ** 2))))

[K = 0] train rmse = 3.8335786493772415, cv_rmse = 3.835029436997053
[K = 1] train rmse = 3.825703772386243, cv_rmse = 3.866156877031664
[K = 2] train rmse = 3.8354324978345815, cv_rmse = 3.818239280896086
[K = 3] train rmse = 3.838217366037447, cv_rmse = 3.8042307589817144
[K = 4] train rmse = 3.819357889849804, cv_rmse = 3.888622155479791


In [50]:
pred = pd.DataFrame(model.predict(test), index=test.index, columns=['target'])

In [51]:
pred.to_csv('./data/sub.csv')